# Load data - RNA molecules with methylated adenosine

In this notebook we are going to download the data from `Zenodo` storing them in suitably organised directories. Then, we can proceed with running the `Tutorial` notebooks.

We remind you to have a look at our paper https://arxiv.org/pdf/2411.07798 and at the documentation https://www.bussilab.org/doc-MDRefine/MDRefine/index.html.

Required assistance may be opened at https://github.com/bussilab/MDRefine/issues.

In [1]:
import os
import numpy as np
import jax.numpy as jnp
import pandas
import matplotlib.pyplot as plt

In [2]:
os.chdir('../')
from MDRefine import load_data

os.chdir('../')

*Molecular dynamics simulations of chemically modified ribonucleotides* - Valerio Piomponi, Mattia Bernetti, Giovanni Bussi

### Download from Zenodo

Let's download data from Zenodo https://zenodo.org/records/6498021 into folder `DATA_alchemical_Zenodo`. There are some data also in `Examples/github_data`.

Then, there are some staffs to do preliminarly in order to get the following structure of folder `DATA_alchemical`, which will be loaded directly through `load_data`.

In the folder `DATA_alchemical` you are going to have:
- a folder `alchemical` which includes `DDGs` (experimental values), `logZs` (log of the partition functions) and `temperature.txt` files;
- several subfolders, each of them corresponding to a different molecular system; in each subfolder you will have `weights.npy` (weights of each frame) and, for the methylated systems, `ff_terms.npy` (force-field correction terms).

The following lines work for **A1** thermodynamic cycle, then you can  repeat the same analysis for A2, A3, A4, A5, B1, B2, B3, B4, B5 by suitably decommenting some code lines

In [3]:
!mkdir DATA_alchemical_Zenodo

the whole list of thermodynamic cycles is:

`['A1', 'A2', 'A3', 'A4', 'A5', 'B1', 'B2', 'B3', 'B4', 'B5']`

In [1]:
cycle_names = ['A1', 'A2', 'A3', 'A4', 'A5']  # , 'B1', 'B2', 'B3', 'B4', 'B5']

In [5]:
%%bash -s "{" ".join(cycle_names)}"

for s in $1
do
    curl -O https://zenodo.org/records/6498021/files/${s}.zip
    mkdir ${s}
    mv ${s}.zip ${s}
    unzip ${s}/${s}.zip -d ${s}
    rm -r ${s}/${s}.zip
    mv ${s} DATA_alchemical_Zenodo

    # remove non-required files (elements not ending with "Aduri")
    cd DATA_alchemical_Zenodo/${s}

    for ELEMENT in $( ls | grep -v "Aduri" ); do
        yes | rm -r $ELEMENT
    done

    cd ../../
done

Archive:  A1/A1.zip
  inflating: A1/md_lam_0.mdp         
  inflating: A1/md_lam_10.mdp        
  inflating: A1/md_lam_11.mdp        
  inflating: A1/md_lam_12.mdp        
  inflating: A1/md_lam_13.mdp        
  inflating: A1/md_lam_14.mdp        
  inflating: A1/md_lam_15.mdp        
  inflating: A1/md_lam_1.mdp         
  inflating: A1/md_lam_2.mdp         
  inflating: A1/md_lam_3.mdp         
  inflating: A1/md_lam_4.mdp         
  inflating: A1/md_lam_5.mdp         
  inflating: A1/md_lam_6.mdp         
  inflating: A1/md_lam_7.mdp         
  inflating: A1/md_lam_8.mdp         
  inflating: A1/md_lam_9.mdp         
   creating: A1/A1_anti_Aduri/
  inflating: A1/A1_anti_Aduri/replica_index.xvg  
  inflating: A1/A1_anti_Aduri/replica_temp.xvg  
  inflating: A1/A1_anti_Aduri/topol.top  
  inflating: A1/A1_anti_Aduri/rerun10_lam15.edr  
  inflating: A1/A1_anti_Aduri/rerun11_lam15.edr  
  inflating: A1/A1_anti_Aduri/rerun12_lam15.edr  
  inflating: A1/A1_anti_Aduri/rerun13_lam15.edr  


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 4923M  100 4923M    0     0  34.9M      0  0:02:20  0:02:20 --:--:-- 34.7M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 7308M  100 7308M    0     0  35.0M      0  0:03:28  0:03:28 --:--:-- 35.0M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 4994M  100 4994M    0     0  35.8M      0  0:02:19  0:02:19 --:--:-- 36.8M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 4963M  100 4963M    0     0  35.4M      0  0:02:19  0:02:19 --:--:-- 32.6M
  % Total    % Received % Xferd  Average Speed   Tim

In [6]:
import panedr
from bussilab import wham

make directory `DATA_alchemical`, in which you are going to store the relevant information from `DATA_alchemical_Zenodo` after some calculations

In [7]:
%%bash

mkdir -p DATA_alchemical
mkdir -p DATA_alchemical/alchemical

- experimental values from `MDRefine/Examples/github_DATA/`

In [8]:
!cp MDRefine/Examples/github_DATA/alchemical/DDGs DATA_alchemical/alchemical/DDGs

In [9]:
pandas.read_csv('DATA_alchemical/alchemical/DDGs')

,name,A1,A2,A3,A4,A5,B1,B2,B3,B4,B5
0,exp. value,6.3,1.7,7.1,-2.5,-1.7,2.5,2.1,5.4,8.6,1.7
1,uncertainty,0.5,0.9,0.9,1.2,0.9,2.1,1.3,1.3,0.8,1.0


- temperature: read it from temperature.txt file (manually)

In [10]:
!cp MDRefine/Examples/github_DATA/alchemical/temperature.txt DATA_alchemical/alchemical/temperature.txt

In [11]:
temperature = 2.476  # 2.476 kJ/mol, namely 298 K

- weights and logZs

In [12]:
weights = {}
dE = {}
logZ = {}
DeltaG = {}

Ene2d_list = []

path_directory = 'DATA_alchemical_Zenodo/'

In [13]:
N = 16

for cycle_name in cycle_names:
   if cycle_name == 'A1': names = ['anti_Aduri', 'syn_Aduri']
   elif cycle_name in ['A2', 'A3']: names = ['ss_Aduri', 'dup_anti_Aduri', 'dup_syn_Aduri']
   elif cycle_name in ['A4', 'A5']: names = ['ss_Aduri', 'dup_Aduri']
   elif cycle_name in ['B1','B2','B3','B4','B5']: names = ['ss_Aduri', 'dup_Aduri']


   for s in names:
      print(cycle_name,s)

      # 1. import energies

      em = path_directory + cycle_name + '/' + cycle_name + '_' + s + '/lam%s/ener_trj_conc.edr'

      Ene2d = []

      for i in range(N):
         df = panedr.edr_to_df(em % str(i))
         Ene2d.append(np.array(df[u'Potential']))
         print(i + 1, '/', N)

      Ene2d = np.array(Ene2d).T
      Ene2d-= np.min(Ene2d)

      # 2. do WHAM

      bias = (Ene2d.T - Ene2d[:,0]).T
      a = wham.wham(bias=bias, T=temperature, maxiter=10000)

      # 3. append weights, dE, logZ

      # 'D' for the duplex structure (for A2, A3, the most common is the anti), 'S' for single-stranded
      # for A1 (methylated adenosine), let's use 'D' for anti and 'S' for syn
      if (s == 'dup_Aduri') or (s == 'dup_anti_Aduri') or (s == 'anti_Aduri'): s1 = 'D'
      elif (s == 'ss_Aduri') or (s == 'syn_Aduri'): s1 = 'S'
      else: s1 = s

      w = np.exp(a['logW'])
      w = w/np.sum(w)

      weights['%s_A%s' % (cycle_name,s1)] = w # A (wild type)

      dE['%s_%s' % (cycle_name, s1)] = bias[:, -1]

      temp = w*np.exp(-dE['%s_%s' % (cycle_name, s1)]/temperature)
      logZ['%s_M%s' % (cycle_name, s1)] = np.log(np.sum(temp)) # logZ for methylated structure

      temp = temp/np.sum(temp)
      weights['%s_M%s' % (cycle_name, s1)] = temp # weights for methylated structure

      # 4. append DeltaG

      # DeltaG = -temperature*(a[u'logZ'][N-1]-a[u'logZ'][0])
      
      # or equivalently:
      DeltaG['%s_%s' % (cycle_name, s1)] = -temperature*logZ['%s_M%s' % (cycle_name, s1)]

      # or equivalently: 
      # DeltaG['%s_%s' % (cycle_name,s1)] = -temperature*np.log(np.sum(weights['%s_A%s' % (cycle_name,s1)]*np.exp(-dE['%s_%s' % (cycle_name,s1)]/temperature)))


A1 anti_Aduri
1 / 16
2 / 16
3 / 16
4 / 16
5 / 16
6 / 16
7 / 16
8 / 16
9 / 16
10 / 16
11 / 16
12 / 16
13 / 16
14 / 16
15 / 16
16 / 16
A1 syn_Aduri
1 / 16
2 / 16
3 / 16
4 / 16
5 / 16
6 / 16
7 / 16
8 / 16
9 / 16
10 / 16
11 / 16
12 / 16
13 / 16
14 / 16
15 / 16
16 / 16
A2 ss_Aduri
1 / 16
2 / 16
3 / 16
4 / 16
5 / 16
6 / 16
7 / 16
8 / 16
9 / 16
10 / 16
11 / 16
12 / 16
13 / 16
14 / 16
15 / 16
16 / 16
A2 dup_anti_Aduri
1 / 16
2 / 16
3 / 16
4 / 16
5 / 16
6 / 16
7 / 16
8 / 16
9 / 16
10 / 16
11 / 16
12 / 16
13 / 16
14 / 16
15 / 16
16 / 16
A2 dup_syn_Aduri
1 / 16
2 / 16
3 / 16
4 / 16
5 / 16
6 / 16
7 / 16
8 / 16
9 / 16
10 / 16
11 / 16
12 / 16
13 / 16
14 / 16
15 / 16
16 / 16
A3 ss_Aduri
1 / 16
2 / 16
3 / 16
4 / 16
5 / 16
6 / 16
7 / 16
8 / 16
9 / 16
10 / 16
11 / 16
12 / 16
13 / 16
14 / 16
15 / 16
16 / 16
A3 dup_anti_Aduri
1 / 16
2 / 16
3 / 16
4 / 16
5 / 16
6 / 16
7 / 16
8 / 16
9 / 16
10 / 16
11 / 16
12 / 16
13 / 16
14 / 16
15 / 16
16 / 16
A3 dup_syn_Aduri
1 / 16
2 / 16
3 / 16
4 / 16
5 / 16
6 / 16
7 / 

In [14]:
for k in weights.keys():
    os.mkdir('DATA_alchemical/' + k)
    np.save('DATA_alchemical/' + k + '/weights.npy', weights[k])

In [15]:
f = open('DATA_alchemical/alchemical/logZs', 'a+')
for k in logZ.keys(): f.write(k + ',' + str(logZ[k]) + '\n')
f.close()

- force-field corrections

load matrix of charges:

$\Delta Q_1,\, \Delta Q_1^2,\, \Delta Q_2,\, \Delta Q_2^2,\,....,\,\Delta Q_5^2,\,\Delta Q_1\cdot \Delta Q_2,\, ... ,\,\Delta Q_1\cdot \Delta Q_5,\, ...,\,\Delta Q_4\cdot \Delta Q_5$

for 20 (random) choices of charges $\Delta Q_1 ,\,...,\,\Delta Q_5$

they correspond to Q1: N6; Q2: H61; Q3: N1; Q4: C10; Q5: H101/2/3

In [16]:
DQs = np.load('MDRefine/Examples/github_DATA/alchemical/DQs.npy')

In [17]:
Tr = {}

for cycle_name in cycle_names:

    if cycle_name == 'A1': names = ['anti_Aduri', 'syn_Aduri']
    elif cycle_name in ['A2', 'A3']: names = ['ss_Aduri', 'dup_anti_Aduri']
    elif cycle_name in ['A4', 'A5']: names = ['ss_Aduri', 'dup_Aduri']
    elif cycle_name in ['B1','B2','B3','B4','B5']: names = ['ss_Aduri', 'dup_Aduri']
    
    for s in names:
        print(cycle_name, s)

        my_path = path_directory + '%s/%s_%s/' % (cycle_name, cycle_name, s)

        # step 1: read energies corresponding to different choices of charges
        ME = []
        for i in range(20):
            em = my_path + 'rerun%i_lam15.edr' % (i+1)
            df = panedr.edr_to_df(em)
            ME.append(np.array(df[u'Potential']))
            print(i)
        ME = np.array(ME)

        # step 2: read E0
        em = my_path + 'lam15/ener_trj_conc.edr'
        df = panedr.edr_to_df(em)
        E0 = np.array(df[u'Potential'])

        # step 3: from dE, dQ to force-field correction terms
        # dE_i = Q_vec_ij * f_j (matrix product)
        
        TrM = []
        inv_mat = np.linalg.inv(np.transpose(DQs))

        for i in range(len(E0)):
            dE = ME[:, i] - E0[i]
            x = inv_mat.dot(dE)
            TrM.append(x)

        # 'D' for the duplex structure (for A2, A3, the most common is the anti), 'S' for single-stranded
        # for A1 (methylated adenosine), let's use 'D' for anti and 'S' for syn
        if (s == 'dup_Aduri') or (s == 'dup_anti_Aduri') or (s == 'anti_Aduri'): s1 = 'D'
        elif (s == 'ss_Aduri') or (s == 'syn_Aduri'): s1 = 'S'
        else: s1 = s

        Tr['%s_M%s' % (cycle_name, s1)] = np.array(TrM)

A1 anti_Aduri
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
A1 syn_Aduri
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
A2 ss_Aduri
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
A2 dup_anti_Aduri
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
A3 ss_Aduri
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
A3 dup_anti_Aduri
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
A4 ss_Aduri
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
A4 dup_Aduri
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
A5 ss_Aduri
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
A5 dup_Aduri
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19


In [18]:
for k in Tr.keys():
    np.save('DATA_alchemical/' + k + '/ff_terms.npy', Tr[k])

$\eta$ angles are in github_DATA (not on Zenodo); append them to ff_terms computed from Zenodo data (related to charges)

In [19]:
for cycle_name in cycle_names:
    for s in ['D', 'S']:
        vec = np.load('DATA_alchemical/%s_M%s/ff_terms.npy' % (cycle_name, s))
        cos_eta = np.load('MDRefine/Examples/github_DATA/alchemical/cos_etas/cos_eta_%s_M%s.npy' % (cycle_name, s))
        vec = np.hstack((vec, cos_eta[:, None]))
        np.save('DATA_alchemical/%s_M%s/ff_terms.npy' % (cycle_name, s), vec)

In [20]:
!rm -r DATA_alchemical_Zenodo

### Load data

Now you have all the required data in the folder `DATA_alchemical` and you can load them with `load_data` function of `MDRefine`

In [21]:
infos = {}

infos['global'] = {}
infos['global']['temperature'] = 2.476
infos['global']['path_directory'] = 'DATA_alchemical'

# names of the thermodynamic cyles and associated molecular systems, in the correct order

cycle_names = ['A1']  # ,'A2','A3','A4','A5']  # ,'B1','B2','B3','B4','B5']

names = {}
for name in cycle_names:
    names[name] = []
    for string in ['AS','AD','MS','MD']:
        names[name].append((name + '_' + string))

infos['global']['cycle_names'] = names
infos['global']['system_names'] = [s2 for s in list(names.values()) for s2 in s]

# force-field correction terms

n_charges = 5

infos['global']['names_ff_pars'] = ['DQ %i' % (i+1) for i in range(n_charges)] + ['cos eta']

columns = []
for i in range(n_charges):
    columns.append('DQ %i' % (i+1))
    columns.append('DQ %i%i' % (i+1,i+1))
for i in range(n_charges):
    for j in range(i+1,n_charges):
        columns.append('DQ %i%i' % (i+1,j+1))
columns.append('cos eta')

# only methylated (M) systems have a force-field correction

for name in infos['global']['system_names']: infos[name] = {}

for name in infos['global']['cycle_names'].keys():
    for s in ['D','S']:
        infos[name + '_M' + s]['ff_terms'] = columns

Define the force-field correction as

\begin{equation}
\Delta U(x) = \sum_{i=1}^5 K_i(x) \Delta Q_i + \sum_{i=1}^5 \sum_{j=i}^5 K_{ij}(x) \Delta Q_i \Delta Q_j - V_\eta \cos\eta_6(x);
\end{equation}

- use `jax.numpy` rather than `numpy` in order to do automatic differentiation;
- the force-field coefficients $\phi$ are: `['DQ %i' for i in range(n_charges)] + ['cos eta']`
- the columns of the force-field correction are

`['DQ 1', 'DQ 11', 'DQ 2', 'DQ 22', 'DQ 3', 'DQ 33', 'DQ 4', 'DQ 44', 'DQ 5', 'DQ 55', 'DQ 12', 'DQ 13', 'DQ 14', 'DQ 15', 'DQ 23', 'DQ 24', 'DQ 25', 'DQ 34', 'DQ 35', 'DQ 45', 'cos eta']`

where 2 numbers (such as `DQ 11` or `DQ 23`) mean the product `DQ 1 * DQ 1` or `DQ 2 * DQ 3`.

These charges correspond to: (Q1: N6; Q2: H61; Q3: N1; Q4: C10; Q5: H101/2/3)

In [22]:
names_charges = ['N6', 'H61', 'N1', 'C10', 'H101/2/3']

In [23]:
def ff_correction(phi, ff_terms):

    n_charges = 5

    phi_vector = []
    for i in range(n_charges):
        phi_vector.extend([phi[i], phi[i]**2])
    for i in range(n_charges):
        for j in range(i+1,n_charges):
            phi_vector.append(phi[i]*phi[j])
    phi_vector.append(-phi[-1])
    phi_vector = jnp.array(phi_vector)

    correction = jnp.matmul(ff_terms, phi_vector)

    return correction

In [24]:
for k in infos['global']['system_names']:
    if k[-2] == 'M': infos[k]['ff_correction'] = ff_correction

In [25]:
data = load_data(infos)

loading data from directory...
loading  A1_AS


loading  A1_AD
loading  A1_MS
loading  A1_MD
done


visualize `data` object

In [26]:
vars(data.properties)

{'system_names': ['A1_AS', 'A1_AD', 'A1_MS', 'A1_MD'],
 'names_ff_pars': ['DQ 1', 'DQ 2', 'DQ 3', 'DQ 4', 'DQ 5', 'cos eta'],
 'cycle_names': {'A1': ['A1_AS', 'A1_AD', 'A1_MS', 'A1_MD']}}

In [27]:
vars(data.mol['A1_MS'])

{'temperature': 2.476,
 'weights': DeviceArray([0.00000000e+000, 0.00000000e+000, 1.33713526e-108, ...,
              3.49185156e-015, 1.78055044e-099, 9.85168166e-021],            dtype=float64),
 'ff_correction': <function __main__.ff_correction(phi, ff_terms)>,
 'f': array([[-4.96519100e+01, -1.38151792e-01, -1.51406786e+02, ...,
          2.67895059e+02, -1.20937458e+03,  9.84882318e-01],
        [-3.98867112e+01, -4.83881329e-01, -1.61957292e+02, ...,
          2.79517133e+02, -1.22971574e+03,  9.93828308e-01],
        [ 1.28829748e+01, -6.39883624e-02, -9.58745191e+01, ...,
          3.03679715e+02, -1.19964908e+03,  9.94782328e-01],
        ...,
        [-1.17374935e+01,  1.90374608e-01, -8.50660721e+01, ...,
          2.30174596e+02, -1.19839557e+03,  9.78575987e-01],
        [-2.92366135e+00, -4.74383497e-02, -1.21207678e+02, ...,
          1.77031228e+02, -1.19014274e+03,  9.99784955e-01],
        [ 2.72489060e+00,  1.42248795e-02, -1.08586519e+02, ...,
          1.38584548e+

force-field correction terms: the following order is implied in `ff_terms.npy`

In [28]:
n_charges = 5
columns = []
for i in range(n_charges):
    columns.append('DQ %i' % (i+1))
    columns.append('DQ %i%i' % (i+1,i+1))
for i in range(n_charges):
    for j in range(i+1,n_charges):
        columns.append('DQ %i%i' % (i+1,j+1))
columns.append('cos eta')

print(columns)

['DQ 1', 'DQ 11', 'DQ 2', 'DQ 22', 'DQ 3', 'DQ 33', 'DQ 4', 'DQ 44', 'DQ 5', 'DQ 55', 'DQ 12', 'DQ 13', 'DQ 14', 'DQ 15', 'DQ 23', 'DQ 24', 'DQ 25', 'DQ 34', 'DQ 35', 'DQ 45', 'cos eta']


In [29]:
pandas.DataFrame(data.mol['A1_MD'].f, columns = columns)

,DQ 1,DQ 11,DQ 2,DQ 22,DQ 3,DQ 33,DQ 4,DQ 44,DQ 5,DQ 55,...,DQ 13,DQ 14,DQ 15,DQ 23,DQ 24,DQ 25,DQ 34,DQ 35,DQ 45,cos eta
0,-64.971763,-0.369189,-193.036466,-0.860319,372.254488,0.021453,-402.717426,-0.456240,-483.793322,-3591.759502,...,0.204762,-0.755635,-1198.668219,461.637943,-0.856726,132.493415,314.638583,-179.944450,-1199.190803,-0.989419
1,-36.861680,0.112033,-228.266966,0.379891,367.954737,-0.245896,-426.504955,-0.395169,-544.641296,-3618.004971,...,-0.170276,-0.114986,-1203.227681,514.610128,-0.075963,125.894241,335.859019,-154.576077,-1206.216901,-0.994984
2,-9.741237,-0.102778,-151.728121,-0.174974,368.018225,-0.116965,-485.212113,-0.401636,-298.747327,-3565.775323,...,0.056460,-0.358373,-1188.091939,493.126934,-0.324271,153.834757,324.601722,-155.636396,-1189.654161,-0.999074
3,-23.380554,-0.134656,-140.835976,-0.305287,366.440820,-0.119104,-334.751580,-0.431187,22.737697,-3414.551457,...,0.106140,-0.426060,-1138.018041,479.168234,-0.416399,160.577397,311.159928,-149.720741,-1139.511416,-0.999357
4,-5.937648,-0.206319,-95.041680,-0.482795,366.137959,-0.064504,-304.141108,-0.376703,-137.692347,-3639.757275,...,0.136286,-0.458487,-1213.749330,497.276130,-0.467213,121.361258,326.301992,-168.805522,-1214.630600,-0.991914
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
160011,-20.199958,0.328535,-156.364218,0.137758,349.173404,-0.080967,-277.607596,-0.317463,-176.108445,-3489.019615,...,0.616035,0.291780,-1164.185146,481.882679,0.165014,139.388203,316.704516,-154.931042,-1164.882082,-0.998520
160012,-21.959617,-0.088238,-126.747990,-0.054939,343.385236,-0.150294,-368.198603,-0.375910,-209.163393,-3593.929224,...,0.000384,-0.358410,-1197.157860,499.693335,-0.225965,123.566694,325.621083,-161.098741,-1198.873484,-0.918453
160013,-39.445614,-0.250645,-135.277098,-0.501048,360.982347,-0.089520,-400.860409,-0.368149,-242.404747,-3821.104001,...,0.055932,-0.545325,-1274.240752,503.545013,-0.492402,117.097418,331.612984,-196.135128,-1274.925486,-0.998902
160014,-44.557677,-0.056004,-203.929256,-0.226455,415.988721,-0.075614,-446.763277,-0.388657,-538.419943,-3568.494427,...,0.141293,-0.280380,-1189.091135,509.670895,-0.351320,130.930304,323.340100,-159.907965,-1190.786733,-0.972010
